# Chocolate Rating Analysis 

## Web scraping data to then process, clean, etc. in the notebook.

In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import chromedriver_autoinstaller
def create_driver(headless=True):
    driver = chromedriver_autoinstaller.install(cwd=True)
    chrome_options = webdriver.ChromeOptions()     
    if headless:
        chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(driver, 
                             chrome_options = chrome_options)
    return driver
driver = create_driver()

<ipython-input-1-097e79a90c76>:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver,


In [2]:
driver.get("http://flavorsofcacao.com/chocolate_database.html")

## Importing Necessary Python Libraries 

In [10]:
div_id = "spryregion1"

html = driver.page_source
soup = BeautifulSoup(html)
div = soup.find('div', {'id':div_id})
# div

table = div.find('table')
table


<table border="1">
<tbody><tr>
<th height="26">REF</th>
<th>Company (Manufacturer)</th>
<th>Company Location</th>
<th>Review Date</th>
<th>Country of Bean Origin</th>
<th>Specific Bean Origin or Bar Name</th>
<th>Cocoa Percent</th>
<th>Ingredients</th>
<th>Most Memorable Characteristics</th>
<th>Rating</th>
</tr>
<tr>
<td>2454</td>
<td>5150</td>
<td>U.S.A.</td>
<td>2019</td>
<td>Madagascar</td>
<td>Bejofo Estate, batch 1</td>
<td>76%</td>
<td>3- B,S,C</td>
<td>cocoa, blackberry, full body</td>
<td>3.75</td>
</tr><tr>
<td>2458</td>
<td>5150</td>
<td>U.S.A.</td>
<td>2019</td>
<td>Dominican Republic</td>
<td>Zorzal, batch 1</td>
<td>76%</td>
<td>3- B,S,C</td>
<td>cocoa, vegetal, savory</td>
<td>3.5</td>
</tr><tr>
<td>2454</td>
<td>5150</td>
<td>U.S.A.</td>
<td>2019</td>
<td>Tanzania</td>
<td>Kokoa Kamili, batch 1 </td>
<td>76%</td>
<td>3- B,S,C</td>
<td>rich cocoa, fatty, bready</td>
<td>3.25</td>
</tr><tr>
<td>797</td>
<td>A. Morin</td>
<td>France</td>
<td>2012</td>
<td>Peru</td>
<td>Per

In [11]:
df = pd.read_html(str(table))[0]

In [12]:
df.head()

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,797,A. Morin,France,2012,Peru,Peru,63%,"4- B,S,C,L","fruity, melon, roasty",3.75
4,797,A. Morin,France,2012,Bolivia,Bolivia,70%,"4- B,S,C,L","vegetal, nutty",3.50


In [13]:
df.to_csv('../../../data/chocolate.csv', index=False)